#### Business Analytics FHDW 2025

# Künstliche Neuronale Netze mit TensorFlow und Keras

Falls *TensorFlow* noch nicht installiert ist:

In [ ]:
! pip install tensorflow

*Keras* ist ein Bestandteil der *TensorFlow*-Bibliothek:

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from dmba import regressionSummary

Als Beispiel betrachten wir die Daten aus `TinyData.csv`. Numerische Nahrungsbestandteile aus *Fett* und *Salz* werden dort der kategorischen Einordnung in *schmeckt* oder *schmeckt nicht* zugeordnet.

In [ ]:
tiny_example_df = pd.read_csv('./Daten/TinyData.csv')
tiny_example_df

Die Daten bereiten wir passend als Prädiktoren und Ziel auf. Wie gewohnt wird aus den Kategorien eine Dummy-Variable *like*.

In [ ]:
predictors = ['fat', 'salt']
outcome = 'acceptance'

X = tiny_example_df[predictors]
y = pd.get_dummies(tiny_example_df[outcome], drop_first=True)
classes = sorted(tiny_example_df[outcome].unique())
y

Nun konstruieren wir ein einfaches Netz aus den bekannten, dicht verknüpften *Multilayer Perceptrons* (MLP). Die dichte Verknüpfung, d. h. alle Inputs der Knoten einer Schicht sind jeweils mit allen Outputs der vorherigen Schicht verbunden, erreichen wir hier durch Verwendung von `tensorflow.keras.layers.Dense`. Ein solcher Dense Layer impliziert diese Eigenschaft, wenn wir ihn mit anderen verknüpfen.

Wir nutzen die *Functional API* von Keras, d. h. die einzelnen Schichten werden als Funktionen definiert und auch funktional miteinander verbunden, indem die Ergebnisse der vorherigen Schicht als Parameter der folgenden Schicht dienen.

Es geht los mit einer Eingabe-Schicht `inputs`, die einen zweidimensionalen Vektor der Fett- und Salz-Werte akzeptiert. Genau genommen ist die erste Schicht damit eigentlich ein Tensor. Dieser wird übergeben an eine "verborgene" Dichte-Schicht `x`, die aus drei Knoten besteht, die durch logistische Sigmoid-Funktionen aktiviert werden. `x` wird an die Ausgabe-Schicht `outputs` übergeben, deren einzelner Knoten die drei Ergebnisse der versteckten Knoten auf den gesuchten Akzeptanzwert reduziert.

Mit den so erzeugten Variablen `inputs` und `outputs` können wir das Modell instanzieren und uns seine Eigenschaften und Struktur ausgeben lassen.

In [ ]:
inputs = keras.Input(shape=(2,), name='Input_fat_and_salt')
x = layers.Dense(3, activation='sigmoid', name='Hidden_Layer')(inputs)
outputs = layers.Dense(1, name='Output_acceptance')(x)

tiny_model = keras.Model(inputs=inputs, outputs=outputs, name="tiny_model")
tiny_model.summary()
keras.utils.plot_model(tiny_model, show_shapes=True, dpi=80)

Das Modell kompilieren wir und trainieren es mit `fit`. Wegen der 10.000 Durchläufe unterdrücken wir die Ausgaben (`verbose`),

In [ ]:
tiny_model.compile(loss='mean_squared_error',
                   optimizer='adam',
                   metrics=['accuracy'])
tiny_model.fit(x=X, y=y, epochs=10000, verbose=0)

In [ ]:
tiny_model.save('./tiny_model.keras') 

Alternativ laden wir das bereits berechnete Modell wieder:

In [ ]:
tiny_model =  tf.keras.models.load_model('./tiny_model.keras')

Die resultierenden Gewichte des Modells, sowie seine Performance bzw. Vorhersagequalität geben wir aus. Da die Zielvariable zwar kategorisch ist, das Netz aber numerische Werte, die wir als Wahrscheinlichkeiten interpretieren können, erzeugt, werten wir die Ergebnisse wie eine Regression aus.

In [ ]:
for layer in tiny_model.layers: 
    print(layer.get_weights())

arr = [(1-a, a) for a in tiny_model.predict(X)]
print(pd.concat([
    tiny_example_df,
    pd.DataFrame(arr, columns=classes)
], axis=1))
print(y-tiny_model.predict(X))
print(regressionSummary(y, tiny_model.predict(X)))

test_scores = tiny_model.evaluate(X, y, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

## Aufgabe

Implementieren Sie ein XOR-Gatter als KNN mit der *Functional API* von Keras. Trainieren Sie das Netz und überprüfen Sie die Gewichte.

`training_bits = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])`

`target_bits = np.array([[0], [1], [1], [0]])`